In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator
import seaborn as sns
import os
import numpy as np
import sys
from pathlib import Path

sys.path.insert(0, str(Path().resolve().parents[1]))
from src.config import *

In [ ]:
# Choose config file
config = get_config("config.yaml")

# Get results path
res_path = get_base_path(config) / config["results_path"]

# Choose where to save plots
plots_path = get_base_path(config) / "plots"
create_clean_dir(plots_path)

In [ ]:
# Style
plt.style.use('seaborn-v0_8-paper')
plt.rcParams.update({
    'font.size': 9,
    'axes.labelsize': 9,
    'axes.titlesize': 9,
    'legend.fontsize': 7,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'lines.linewidth': 0.8,
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'axes.edgecolor': 'black',
    'axes.linewidth': 0.8,
    'text.usetex': True
})

# Colors
bound_color = "#ff441a"     # red
BN_color = "#3934fe"        # blue
CN_color = "#28bd6b"        # green
alpha = 0.2

In [ ]:
# Plot BN and bound (semilogx)
def plot_bn_bound(exp: str, ax):

    # Import results
    results = os.listdir(res_path)
    r_path = [r for r in results if f"{exp}-ess1.csv" in r][0]
    result = pd.read_csv(f'{res_path}/{r_path}')
    error = result["error"]
    bound = result["power_bound"]
    bn_cols = [c for c in result.columns if "BN" in c]
    bn_mean = result.loc[:,bn_cols].mean(axis=1)
    bn_max = result.loc[:,bn_cols].max(axis=1)

    # Plot bound
    ax.semilogx(error, bound, '^', color=bound_color, label='Theoretical estimate', markersize=4, zorder=4)

    # Plot BN (avg-max)
    ax.fill_between(error, bn_mean, bn_max, color=BN_color, alpha=alpha, zorder=2)
    ax.semilogx(error, bn_mean, '-', color=BN_color, label='BN', zorder=3)

    # Title and axes
    ax.set_xlabel('Error')
    ax.set_ylabel('Power')
    ax.set_xlim(10e-5*0.5, 1)
    ax.set_xticks([10e-4, 10e-3, 10e-2, 10e-1, 1])
    ax.set_xticklabels(["$10^{-4}$", "$10^{-3}$", "$10^{-2}$", "$10^{-1}$", "$10^{0}$"])
    ax.xaxis.set_minor_locator(LogLocator(base=10.0, subs='auto'))
    ax.tick_params(axis='x', which='minor', length=2, width=0.5)
    ax.tick_params(axis='x', which='major', length=3, width=0.9)
    ax.set_ylim(-0.08, 1.08)
    ax.tick_params(axis='y', which='major', length=3, width=0.9)
    ax.grid(True, which='major', linestyle='-', linewidth=0.5, color='#bfbfbf', zorder=1)

# Plot CN (for a given ess)
def plot_cn(exp, ax, ess: int, color: str, type: str):

    # Import results
    results = os.listdir(res_path)
    r_path = [r for r in results if f"{exp}-ess{ess}.csv" in r][0]
    result = pd.read_csv(f'{res_path}/{r_path}')
    error = result["error"]
    cn_cols = [c for c in result.columns if "CN" in c]
    cn_mean = result.loc[:,cn_cols].mean(axis=1)
    cn_max = result.loc[:,cn_cols].max(axis=1)

    # Plot CN (avg-max)
    ax.fill_between(error, cn_mean, cn_max, color=color, alpha=alpha, zorder=2)
    ax.semilogx(error, cn_mean, type, color=color, label=f'CN, $S={ess}$', zorder=3)

    # Legend
    if exp == "exp0":
        ax.legend(loc='best', frameon=True, fancybox=False, framealpha=1, facecolor='#e6e6e6', edgecolor='#8c8c8c')

# Plot title function
def get_title(exp: str):
    with open(f'{res_path}/exp_meta.txt', "r") as meta:
        for row in meta:
            if exp in row:
                pieces = row.split()
                n = int(pieces[pieces.index('Nodes:') + 1])
                e = int(pieces[pieces.index('Edges:') + 1])
                c = int(pieces[pieces.index('Complexity:') + 1])
                return n, e, c

In [ ]:
# Layout 4x3
fig, axes = plt.subplots(4, 3, figsize=(10, 9.5))
# fig.suptitle("Power vs Error", fontsize=18)

exps = ["exp0", "exp1", "exp2",
        "exp3", "exp4", "exp5",
        "exp6", "exp7", "exp8", 
        "exp9", "exp10", "exp11"]
ess = [1, 1000]

# Loop over subplots
for i, ax in enumerate(axes.flat):
    plot_bn_bound(f"{exps[i]}", ax)
    plot_cn(f"{exps[i]}", ax, ess = ess[0], color = CN_color, type="-")
    plot_cn(f"{exps[i]}", ax, ess = ess[1], color = CN_color, type="--")
    (n, e, c) = get_title(exps[i])
    ax.set_title(f"Nodes: {n}, Edges: {e}, Complexity: {c}")

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()
fig.savefig(f"{plots_path}/results_logx.pdf", dpi=1200, bbox_inches='tight', transparent=False)